In [ ]:
import pandas as pd
import os
import numpy as np

#合併資料，只取ShopMemberId, SalePageId, SalePageTitle, Cluster四個欄位
def merge_salepagedata_cluster(
    salepagedata_path, 
    cluster_path, 
    output_path):
    
    # 讀取 SalePage 資料
    try:
        salepagedata_df = pd.read_csv(salepagedata_path, dtype=str)
    except Exception as e:
        print(f"讀取 {salepagedata_path} 時發生錯誤：{e}")
        return

    # 讀取 RFM Cluster 資料
    try:
        cluster_df = pd.read_csv(cluster_path, dtype=str)
    except Exception as e:
        print(f"讀取 {cluster_path} 時發生錯誤：{e}")
        return

    # 合併資料：以 ShopMemberId 為鍵合併
    merged_df = pd.merge(
        salepagedata_df[['ShopMemberId', 'SalePageId', 'SalePageTitle']],
        cluster_df[['ShopMemberId', 'Cluster']],
        on='ShopMemberId',
        how='left'  
        # inner 合併，只有兩邊都有的才會保留
        # left 合併，保留 salepage 中所有資料（即使沒有對應的 Cluster）
        # outer
        # right 合併，保留 RFM 中所有資料（即使沒有對應的 salepa資料）
    )

    # 輸出合併後的結果
    try:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"已輸出：{output_path}，共 {len(merged_df)} 筆資料")
    except Exception as e:
        print(f"儲存 {output_path} 時發生錯誤：{e}")
    
    
# 切割資料：依照 cluster 值切成5個檔案（包含 NaN）
def split_csv_by_cluster(input_path, output_dir, cluster_column="Cluster", cluster_values=[1, 2, 3, 0, "NaN"]):
    # 讀取資料
    try:
        df = pd.read_csv(input_path)
    except Exception as e:
        print(f"讀取檔案時發生錯誤：{e}")
        return

    # 檢查欄位是否存在
    if cluster_column not in df.columns:
        print(f"找不到欄位：{cluster_column}")
        return

    # 建立資料夾
    os.makedirs(output_dir, exist_ok=True)

    # 將非空的 cluster 欄位轉為 int（避免多次轉型）
    df_non_null = df[df[cluster_column].notna()].copy()
    try:
        df_non_null[cluster_column] = df_non_null[cluster_column].astype(int)
    except Exception as e:
        print(f"{cluster_column} 欄位轉換為整數時失敗：{e}")
        return

    # 處理每一個 cluster 值
    for value in cluster_values:
        if value == "NaN":
            cluster_df = df[df[cluster_column].isna()].copy()
            filename = "split_cluster_NaN.csv"
        else:
            cluster_df = df_non_null[df_non_null[cluster_column] == value].copy()
            filename = f"split_cluster_{value}.csv"

        output_path = os.path.join(output_dir, filename)
        cluster_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"Cluster {value}：儲存 {len(cluster_df)} 筆到 {output_path}")


#移除包含特定關鍵字的資料
def remove_by_saletitle_keywords(input_path, keywords, output_path=None):
    import pandas as pd

    try:
        df = pd.read_csv(input_path)
    except Exception as e:
        print(f"讀取檔案失敗：{e}")
        return

    if "SalePageTitle" not in df.columns or "SalePageId" not in df.columns:
        print("找不到 'SalePageTitle' 或 'SalePageId' 欄位")
        return

    # 建立一個總集合來收集所有要移除的 SalePageId
    ids_to_remove = set()

    for keyword in keywords:
        matched_ids = df.loc[df["SalePageTitle"].str.contains(keyword, na=False), "SalePageId"]
        print(f"關鍵字 '{keyword}'：共 {len(matched_ids)} 筆 SalePageId 被標記移除")
        ids_to_remove.update(matched_ids)

    original_count = len(df)
    df_filtered = df[~df["SalePageId"].isin(ids_to_remove)]
    removed_count = original_count - len(df_filtered)

    if output_path is None:
        output_path = input_path

    df_filtered.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"已移除總共 {removed_count} 筆資料，輸出到：{output_path}")


In [ ]:
# merge_salepagedata_cluster(
#     'C:/Users/user/Desktop/Ordersplit_Output2/filtered_salepage_data.csv',
#     'C:/Users/user/Desktop/RFM_Cluster.csv',
#     'C:/Users/user/Desktop/output/merged_result_left.csv'
# )
f = pd.read_csv("C:/Users/user/Desktop/output/merged_result_left.csv")
len(f)
print(f.head(10))
print(f.tail(10))

In [ ]:
# split_csv_by_cluster(
#     input_path="C:/Users/user/Desktop/output/merged_result_left.csv",
#     output_dir="C:/Users/user/Desktop/output"
# )
f = pd.read_csv("C:/Users/user/Desktop/output/split_cluster_2.csv")
print(len(f))
# print(f.head(10))
# print(f.tail(10))
print(f.iloc[-801005:-801000])

In [ ]:
# remove_by_saletitle_keywords(
#     "C:/Users/user/Desktop/output/split_cluster_2.csv",
#     keywords=["入點","贈品-","贈品", "贈品:","-贈品","周年慶官網折價券","點數","匯入帳號","購物金"],
#     output_path="C:/Users/user/Desktop/output/split_cluster_2_filtered.csv"
# )
f = pd.read_csv("C:/Users/user/Desktop/output/split_cluster_2_filtered.csv")
print(len(f))
print(f.iloc[1:5])

In [ ]:
#檢查有哪些關鍵字可以刪除
keyword = "發放"
filtered = f[f["SalePageTitle"].str.contains(keyword, na=False)]
total_count = len(filtered)
print(f"符合關鍵字",keyword,f"的 SalePageTitle 總筆數（含重複）有 {total_count} 筆")

if not filtered.empty:
    # 印出前5筆 SalePageTitle 與 Cluster
    print(filtered[["SalePageId","SalePageTitle"]].head(66))
    
    # 印出符合條件的 SalePageTitle 不重複的值（unique）
    unique_titles = filtered["SalePageTitle"].unique()
    print("Unique SalePageTitle:")
    for title in unique_titles[:66]:  
        print(title)
else:
    print(f"找不到包含「{keyword}」的資料")
    

    
# "贈品"總筆數（含重複）有 34161 筆
# "贈品-"總筆數（含重複）有 24743 筆
# "贈品:"總筆數（含重複）有 2736 筆
# "-贈品"總筆數（含重複）有 163 筆
# "周年慶官網折價券"總筆數（含重複）有 928 筆
# "歸戶"有 614 筆
# "點數"有 556 筆
# "匯入帳號"有 218 筆
# "購物金"有 964 筆


In [ ]:
import pandas as pd

# 檔案路徑
main_path = "C:/Users/user/Desktop/output/split_cluster_2_filtered.csv"
desc_path = "C:/Users/user/Desktop/91APPDataset/SalePage.csv"
output_path = "C:/Users/user/Desktop/output/split_cluster_2_with_desc.csv"

# 讀取主資料與說明資料
main_df = pd.read_csv(main_path)
desc_df = pd.read_csv(desc_path)

# 合併資料：根據 SalePageId
merged_df = pd.merge(main_df, desc_df[["SalePageId", "SaleProductDescShortContent"]],
                     on="SalePageId", how="left")

# 儲存結果
merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"合併完成，共有 {len(merged_df)} 筆資料，儲存到：{output_path}")


In [ ]:
import pandas as pd

#另存一個只有title和description的檔案
def keep_only_title_and_description(input_path, output_path):
    try:
        df = pd.read_csv(input_path)
    except Exception as e:
        print(f"讀取檔案失敗：{e}")
        return

    # 僅保留需要的欄位
    if "SalePageTitle" not in df.columns or "SaleProductDescShortContent" not in df.columns:
        print("必要欄位不存在")
        return

    df_reduced = df[["SalePageTitle", "SaleProductDescShortContent"]]

    try:
        df_reduced.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"成功儲存只包含標題與描述的資料到：{output_path}")
    except Exception as e:
        print(f"儲存檔案失敗：{e}")



In [ ]:
keep_only_title_and_description(
    input_path="C:/Users/user/Desktop/output/split_cluster_2_with_desc.csv",
    output_path="C:/Users/user/Desktop/output/title_desc_only.csv"
)

In [ ]:
import pandas as pd
import re


f = pd.read_csv("C:/Users/user/Desktop/output/title_desc_only.csv")

# 移除 SaleProductDescShortContent 欄位開頭的 "一串數字 + 換行符號"
f["SaleProductDescShortContent"] = f["SaleProductDescShortContent"].str.replace(
    r"^\d+\n", "", regex=True)


f.to_csv("C:/Users/user/Desktop/title_desc_only_filtered.csv", index=False, encoding='utf-8')

In [ ]:
f = pd.read_csv("C:/Users/user/Desktop/title_desc_only_filtered.csv")
print(f.loc[100:102])